<h1><center>K Nearest Neighbor </center></h1>
<center>Fatma Ridaoui 1950044013</center>

In [82]:
import random
import pandas as pd
import numpy as np
from math import sqrt
from sklearn import preprocessing, neighbors, cross_decomposition, svm, model_selection
from sklearn.model_selection import train_test_split,KFold
from matplotlib import pyplot as plt

I have downloaded bank.zip and used "bank.csv" inside 

In [2]:
bank_df= pd.read_csv('bank.csv',sep=";")

In [3]:
bank_df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


##  PREPROCESSING

In [4]:
bank_df.get('marital')

0       married
1       married
2        single
3       married
4       married
         ...   
4516    married
4517    married
4518    married
4519    married
4520     single
Name: marital, Length: 4521, dtype: object

In [5]:
dfMarital = pd.get_dummies(bank_df['marital'],drop_first=True)
dfMarital.head()

,married,single
0,1,0
1,1,0
2,0,1
3,1,0
4,1,0


In [6]:
bank_df = pd.concat([dfMarital,bank_df],axis=1)
bank_df.head()

,married,single,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,1,0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,1,0,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,0,1,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,1,0,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,1,0,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [7]:
#Now we need to drop the marital status
bank_df.drop('marital',axis=1,inplace=True)

In [8]:
bank_df.head()
#Marital stat col is dropped

,married,single,age,job,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,1,0,30,unemployed,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,1,0,33,services,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,0,1,35,management,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,1,0,30,management,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,1,0,59,blue-collar,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [9]:
dfLoan = pd.get_dummies(bank_df['loan'],drop_first=True)
dfLoan.rename(columns= {'yes': 'Loan'}, inplace = True)
dfLoan.head()
#Loan:yes 1, no 0

,Loan
0,0
1,1
2,0
3,1
4,0


In [10]:
bank_df = pd.concat([dfLoan,bank_df],axis=1)
bank_df.head()

,Loan,married,single,age,job,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,0,1,0,30,unemployed,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,1,1,0,33,services,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,0,0,1,35,management,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,1,1,0,30,management,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,0,1,0,59,blue-collar,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [11]:
bank_df.drop('loan',axis=1,inplace=True)

In [12]:
bank_df.head()

,Loan,married,single,age,job,education,default,balance,housing,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,0,1,0,30,unemployed,primary,no,1787,no,cellular,19,oct,79,1,-1,0,unknown,no
1,1,1,0,33,services,secondary,no,4789,yes,cellular,11,may,220,1,339,4,failure,no
2,0,0,1,35,management,tertiary,no,1350,yes,cellular,16,apr,185,1,330,1,failure,no
3,1,1,0,30,management,tertiary,no,1476,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,0,1,0,59,blue-collar,secondary,no,0,yes,unknown,5,may,226,1,-1,0,unknown,no


In [13]:
#default: has credit in default? (categorical: 'no','yes','unknown')
dfDefault = pd.get_dummies(bank_df['default'],drop_first=True)
dfDefault.head()

,yes
0,0
1,0
2,0
3,0
4,0


In [14]:
bank_df = pd.concat([dfDefault,bank_df],axis=1)
bank_df.drop('default',axis=1,inplace=True)
bank_df.rename(columns= {'yes': 'default'}, inplace = True)
bank_df.head()

,default,Loan,married,single,age,job,education,balance,housing,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,0,0,1,0,30,unemployed,primary,1787,no,cellular,19,oct,79,1,-1,0,unknown,no
1,0,1,1,0,33,services,secondary,4789,yes,cellular,11,may,220,1,339,4,failure,no
2,0,0,0,1,35,management,tertiary,1350,yes,cellular,16,apr,185,1,330,1,failure,no
3,0,1,1,0,30,management,tertiary,1476,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,0,0,1,0,59,blue-collar,secondary,0,yes,unknown,5,may,226,1,-1,0,unknown,no


In [15]:
dfHous = pd.get_dummies(bank_df['housing'],drop_first=True)
bank_df = pd.concat([dfHous,bank_df],axis=1)
bank_df.drop('housing',axis=1,inplace=True)
bank_df.rename(columns= {'yes': 'housing'}, inplace = True)
bank_df.head()

,housing,default,Loan,married,single,age,job,education,balance,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,0,0,0,1,0,30,unemployed,primary,1787,cellular,19,oct,79,1,-1,0,unknown,no
1,1,0,1,1,0,33,services,secondary,4789,cellular,11,may,220,1,339,4,failure,no
2,1,0,0,0,1,35,management,tertiary,1350,cellular,16,apr,185,1,330,1,failure,no
3,1,0,1,1,0,30,management,tertiary,1476,unknown,3,jun,199,4,-1,0,unknown,no
4,1,0,0,1,0,59,blue-collar,secondary,0,unknown,5,may,226,1,-1,0,unknown,no


In [16]:
dfContact = pd.get_dummies(bank_df['contact'],drop_first=True) #Telephone, unknown
bank_df = pd.concat([dfContact,bank_df],axis=1)
bank_df.drop('contact',axis=1,inplace=True)
bank_df.rename(columns= {'telephone': 'cont_tph'}, inplace = True)
bank_df.rename(columns= {'unknown': 'cont_unk'}, inplace = True)
bank_df.head()

,cont_tph,cont_unk,housing,default,Loan,married,single,age,job,education,balance,day,month,duration,campaign,pdays,previous,poutcome,y
0,0,0,0,0,0,1,0,30,unemployed,primary,1787,19,oct,79,1,-1,0,unknown,no
1,0,0,1,0,1,1,0,33,services,secondary,4789,11,may,220,1,339,4,failure,no
2,0,0,1,0,0,0,1,35,management,tertiary,1350,16,apr,185,1,330,1,failure,no
3,0,1,1,0,1,1,0,30,management,tertiary,1476,3,jun,199,4,-1,0,unknown,no
4,0,1,1,0,0,1,0,59,blue-collar,secondary,0,5,may,226,1,-1,0,unknown,no


In [17]:
job = pd.get_dummies(bank_df['job'],drop_first=True)
job.drop('unknown',axis=1,inplace=True)
bank_df = pd.concat([job,bank_df],axis=1)
bank_df.drop('job',axis=1,inplace=True)
bank_df.head()

,blue-collar,entrepreneur,housemaid,management,retired,self-employed,services,student,technician,unemployed,...,education,balance,day,month,duration,campaign,pdays,previous,poutcome,y
0,0,0,0,0,0,0,0,0,0,1,...,primary,1787,19,oct,79,1,-1,0,unknown,no
1,0,0,0,0,0,0,1,0,0,0,...,secondary,4789,11,may,220,1,339,4,failure,no
2,0,0,0,1,0,0,0,0,0,0,...,tertiary,1350,16,apr,185,1,330,1,failure,no
3,0,0,0,1,0,0,0,0,0,0,...,tertiary,1476,3,jun,199,4,-1,0,unknown,no
4,1,0,0,0,0,0,0,0,0,0,...,secondary,0,5,may,226,1,-1,0,unknown,no


In [18]:
edc = pd.get_dummies(bank_df['education'],drop_first=True)

bank_df = pd.concat([job,bank_df],axis=1)
bank_df.drop('education',axis=1,inplace=True)
bank_df.head()

,blue-collar,entrepreneur,housemaid,management,retired,self-employed,services,student,technician,unemployed,...,age,balance,day,month,duration,campaign,pdays,previous,poutcome,y
0,0,0,0,0,0,0,0,0,0,1,...,30,1787,19,oct,79,1,-1,0,unknown,no
1,0,0,0,0,0,0,1,0,0,0,...,33,4789,11,may,220,1,339,4,failure,no
2,0,0,0,1,0,0,0,0,0,0,...,35,1350,16,apr,185,1,330,1,failure,no
3,0,0,0,1,0,0,0,0,0,0,...,30,1476,3,jun,199,4,-1,0,unknown,no
4,1,0,0,0,0,0,0,0,0,0,...,59,0,5,may,226,1,-1,0,unknown,no


In [19]:
label = pd.get_dummies(bank_df['y'],drop_first=True)

bank_df = pd.concat([label,bank_df],axis=1)
bank_df.drop('y',axis=1,inplace=True)
bank_df.rename(columns= {'yes': 'y'}, inplace = True)
bank_df.head()

,y,blue-collar,entrepreneur,housemaid,management,retired,self-employed,services,student,technician,...,single,age,balance,day,month,duration,campaign,pdays,previous,poutcome
0,0,0,0,0,0,0,0,0,0,0,...,0,30,1787,19,oct,79,1,-1,0,unknown
1,0,0,0,0,0,0,0,1,0,0,...,0,33,4789,11,may,220,1,339,4,failure
2,0,0,0,0,1,0,0,0,0,0,...,1,35,1350,16,apr,185,1,330,1,failure
3,0,0,0,0,1,0,0,0,0,0,...,0,30,1476,3,jun,199,4,-1,0,unknown
4,0,1,0,0,0,0,0,0,0,0,...,0,59,0,5,may,226,1,-1,0,unknown


In [20]:
dfPout= pd.get_dummies(bank_df['poutcome'],drop_first=True) #previous outcome: other	success	unknown
bank_df = pd.concat([dfPout,bank_df],axis=1)
bank_df.drop('poutcome',axis=1,inplace=True)
bank_df.rename(columns= {'other': 'pout_other'}, inplace = True)
bank_df.rename(columns= {'success': 'pout_success'}, inplace = True)
bank_df.rename(columns= {'unknown': 'pout_unk'}, inplace = True)
bank_df.head()

,pout_other,pout_success,pout_unk,y,blue-collar,entrepreneur,housemaid,management,retired,self-employed,...,married,single,age,balance,day,month,duration,campaign,pdays,previous
0,0,0,1,0,0,0,0,0,0,0,...,1,0,30,1787,19,oct,79,1,-1,0
1,0,0,0,0,0,0,0,0,0,0,...,1,0,33,4789,11,may,220,1,339,4
2,0,0,0,0,0,0,0,1,0,0,...,0,1,35,1350,16,apr,185,1,330,1
3,0,0,1,0,0,0,0,1,0,0,...,1,0,30,1476,3,jun,199,4,-1,0
4,0,0,1,0,1,0,0,0,0,0,...,1,0,59,0,5,may,226,1,-1,0


We still have **job, education, balance, day, month,duration, campaign, pdays,previous,poutcome,y** <br>
I did not use label encoding enc because it wouldnt be effective as they have more than 4 unique values 
which will be 4 new different attributes...<br>
**example:** 
job=('admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')

In [21]:
cal = { "jan":1,"feb":2,"mar":3,"apr":4,"may":5,"jun":6,"jul":7,"aug":8,"sep":9,"oct":10,"nov":11,"dec":12 }
month = pd.DataFrame(columns=["month"])

for item in bank_df.month:
    if item in cal.keys():
        month = month.append({"month":cal[item]} ,ignore_index=True )
month

,month
0,10
1,5
2,4
3,6
4,5
...,...
4516,7
4517,5
4518,8
4519,2


In [22]:
bank_df.drop('month',axis=1,inplace=True)
bank_df = pd.concat([month,bank_df],axis=1)
bank_df.head()

,month,pout_other,pout_success,pout_unk,y,blue-collar,entrepreneur,housemaid,management,retired,...,Loan,married,single,age,balance,day,duration,campaign,pdays,previous
0,10,0,0,1,0,0,0,0,0,0,...,0,1,0,30,1787,19,79,1,-1,0
1,5,0,0,0,0,0,0,0,0,0,...,1,1,0,33,4789,11,220,1,339,4
2,4,0,0,0,0,0,0,0,1,0,...,0,0,1,35,1350,16,185,1,330,1
3,6,0,0,1,0,0,0,0,1,0,...,1,1,0,30,1476,3,199,4,-1,0
4,5,0,0,1,0,1,0,0,0,0,...,0,1,0,59,0,5,226,1,-1,0


##  Part 1 : KNN + Euclidean

In [31]:
def euclidean_distance(x, y):
    sum = 0.0
    X= np.array(bank_df.iloc[int(x)])
   
    for i in range(len(X)):
        #print(i)
        sum += (X[i]-y[i])**2
    return sqrt(sum)

In [32]:
def findKPoints(dataset,k):
    k_values = list()
    rand_Num = 0
    while True:  
        rand_Num =random.randrange(0,len(dataset)-1)
        if (rand_Num not in k_values):
            k_values.append(rand_Num)
            if(len(k_values)==k ):  
                return k_values  

In [47]:
def findNearestNeighbor(dataset,k_list):
    for each in np.array(dataset): 
        minn = 0
        neighborhood = list()
        for point in k_list:
            minn = euclidean_distance(point,each);
            neighborhood.append((minn,point))
    sorted_neighb = sorted(neighborhood)    
    return sorted_neighb

In [48]:
k=5
randk_list = findKPoints(np.array(bank_df),k)
#print(randk_list)
neigh =findNearestNeighbor(bank_df,randk_list)
print(neigh)
k_nearest_labels = [np.array(bank_df["y"])[i] for distance,i in neigh]

[(593.5831870934351, 263), (847.4939527807853, 1373), (879.3497597657032, 3347), (1070.4676548126058, 2220), (1159.760751189658, 119)]


In [49]:
k_nearest_labels

[0, 0, 0, 1, 0]

In [50]:
neigh

[(593.5831870934351, 263),
 (847.4939527807853, 1373),
 (879.3497597657032, 3347),
 (1070.4676548126058, 2220),
 (1159.760751189658, 119)]

In [52]:
print(np.array(bank_df["y"])[263],
      np.array(bank_df["y"])[1373],
      np.array(bank_df["y"])[3347],
      np.array(bank_df["y"])[2220],
      np.array(bank_df["y"])[119]
      )

0 0 0 1 0


# Part 2 : KNN + Manhattan

In [54]:
def manhattan_distance(x, y):
    x=np.array(bank_df.iloc[int(x)])
    return sum(abs(a-b) for a,b in zip(x,y))


In [56]:
def findNearestNeighbor(dataset,k_list):
    for each in np.array(dataset): 
        minn = 0
        neighborhood = list()
        for point in k_list:
            minn = manhattan_distance(point,each);
            neighborhood.append((minn,point))
    sorted_neighb = sorted(neighborhood)    
    return sorted_neighb

In [57]:
k=5
randk_list = findKPoints(np.array(bank_df),k)
#print(randk_list)
neigh =findNearestNeighbor(bank_df,randk_list)
print(neigh)
k_nearest_labels = [np.array(bank_df["y"])[i] for distance,i in neigh]

[(779, 3946), (1023, 3297), (1429, 407), (1474, 920), (2948, 282)]


In [58]:
k_nearest_labels

[0, 1, 0, 0, 0]

In [59]:
neigh

[(779, 3946), (1023, 3297), (1429, 407), (1474, 920), (2948, 282)]

In [60]:
for each in neigh:
    print(np.array(bank_df["y"])[each[1]])

0
1
0
0
0


# Part 3

In [83]:
from sklearn.svm import SVR

In [27]:
X= bank_df.drop(["y"],1)
y= bank_df["y"]

X_train, X_test,y_train,y_test=train_test_split(X,y,test_size= 0.2)

In [69]:
clf = svm.SVC()
clf.fit(X_train, y_train)

SVC()

In [70]:
accuracy = clf.score(X_test, y_test)
print(accuracy)

0.8850828729281768


In [71]:
kf = KFold(n_splits=2)
kf.get_n_splits(X)

2

In [73]:
kf

KFold(n_splits=2, random_state=None, shuffle=False)